In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install konlpy

  Using cached urllib3-1.26.7-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 

In [2]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time

from konlpy.tag import Okt
from collections import Counter

In [6]:
id = 'your id'
password = 'your password'

In [4]:
import pandas as pd

df = pd.DataFrame(columns = ['text'])

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent={Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36}")
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

url = 'https://everytime.kr/login'

driver.get(url)

driver.find_element_by_name("userid").send_keys(id)

driver.find_element_by_name("password").send_keys(password)

driver.find_element_by_tag_name("input").send_keys(Keys.RETURN)

time.sleep(2)
driver.find_element_by_xpath('//*[@id="submenu"]/div[1]/div[2]/ul/li[1]//a').click()

time.sleep(0.5)

res = driver.page_source
soup = BeautifulSoup(res,"html.parser")
data_name = soup.select('#container > div.wrap.articles > article > a > h2')
data_text = soup.select('#container > div.wrap.articles > article > a > p')

for name,text in zip(data_name,data_text):
    df.loc[df.shape[0]] = [name.get_text() + ' ' + text.get_text()]  
 
time.sleep(1)
def next_page():
    driver.find_element_by_css_selector('#container > div.wrap.articles > div.pagination > a.next').click()
    time.sleep(0.5)
    
    res = driver.page_source
    soup = BeautifulSoup(res,"html.parser")
    data_name = soup.select('#container > div.wrap.articles > article > a > h2')
    data_text = soup.select('#container > div.wrap.articles > article > a > p')   
    for name,text in zip(data_name,data_text):
        df.loc[df.shape[0]] = [name.get_text() + ' ' + text.get_text()]      
time.sleep(0.5)
for page_roof in range(10):
    next_page()
driver.quit()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_ele

In [5]:
count_dic={}

for text in df.text:
  twitter = Okt()
  noun = twitter.nouns(str(text))
  count_dic.update(Counter(noun))

noun_list = Counter(count_dic).most_common(10)
noun_list

[('미안', 8),
 ('갤러리', 6),
 ('버스', 3),
 ('여친', 3),
 ('남초', 3),
 ('걱정', 3),
 ('알바', 3),
 ('꿈', 3),
 ('수료증', 3),
 ('남', 2)]